## Count amenities near exits
* For each exit, buffer a distance of 1 mile
* Count the # of amenities falling within each buffer
* Add the count to the exits geodataframe
* Export the geodataframe to a shapefile

In [ ]:
#Import packages
import geopandas as gpd

In [ ]:
#Read in exit features and amenity features
exits_gdf = gpd.read_file('../Data/MJBA/Exits.shp')
amenities_gdf = gpd.read_file('../Data/OSM/OSM_amenities.shp')

In [ ]:
#Project each to UTM Zone 17
exits_gdf = exits_gdf.to_crs(32617)
amenities_gdf = amenities_gdf.to_crs(32617)

---
#### Proof of concept: 
Extract # of amenities within buffer of a given point

In [ ]:
theExit = exits_gdf.at[56,'geometry']

In [ ]:
theExit_search = theExit.buffer(1609.34)

In [ ]:
amenities_gdf.geometry.within(theExit_search).sum()

We see that the selected exit feature has two amenities within the search radius. 

---
#### Implementation

Now let's implement that concept in a function and then apply that function to all exit features

In [ ]:
def count_amenities(thePoint,search_radius=1609.34):
    #buffer the point
    thePoly = thePoint.buffer(search_radius)
    #count amenities
    theAmemityCount = amenities_gdf.geometry.within(thePoly).sum()
    #return the count
    return theAmemityCount

In [ ]:
#Apply the funciton to all exit point features
exits_gdf['amenities'] = exits_gdf['geometry'].apply(count_amenities)

In [ ]:
#Export the exits as a feature class
exits_gdf.to_file('../Data/Processed/exits_amenity_counts.shp')

###### Bonus: Quick visualization
* First, a static map with `contextily`

In [ ]:
#Import contextily 
import contextily as ctx

In [ ]:
#Transform to web mercator (to match contextily base maps)
exits_gdf_wm = exits_gdf.to_crs(3857)

In [ ]:
#Plot the amenities to an axis called 'ax'
ax =exits_gdf_wm.plot('amenities',
                      cmap='coolwarm',
                      markersize= exits_gdf['amenities']*10,
                      alpha=0.9,
                      figsize=(20,10));
# Add basemap
ctx.add_basemap(ax)

* Next an interactive map with mplleaflet

In [ ]:
#import matplotlib.pyplot as plt
import mplleaflet

In [ ]:
#Plot the amenities to an axis called 'ax'
ax =exits_gdf.to_crs(4326).plot('amenities',
                      cmap='coolwarm',
                      markersize= exits_gdf['amenities']*10,
                      alpha=0.9,
                      figsize=(20,10));
# 2. Convert plot to a web map and show in a new page
mplleaflet.show(fig=ax.figure)